In [81]:
import geopandas as gpd
import holoviews as hv
import hvplot.pandas
import numpy as np
import pandas as pd
import cenpy
import pygris
from matplotlib import pyplot as plt

import requests

In [87]:
url = "https://mapsdep.nj.gov/arcgis/rest/services/Features/Land/MapServer/65/query"

params = {
    "where": "FUNDING_TYPE = 'Blue Acres Program'",  # Give me all rows
    "outFields": "*",  # All fields
    "f": "geojson",  # GeoJSON format
    "outSR": "4326",  # The desired output CRS
}

r = requests.get(url, params=params)

json = r.json()

features = json["features"]

blueAcres = gpd.GeoDataFrame.from_features(features, crs="EPSG:4326")

blueAcres

,geometry,OBJECTID,COUNTY,MUNICIPALITY,PAMS_PIN,PCL_MUN,PCLBLOCK,PCLLOT,PCLQCODE,GISACRES,...,ACCESS_TYPE,FUNDING_TYPE,ENCUMBRANCE_STATUS,FEE_SIMPLE_NAME,FEE_SIMPLE_ALTRNT_NAME,PCL_GUID,LAST_EDITED_DATE,GLOBALID,SHAPE_Length,SHAPE_Area
0,"POLYGON ((-74.26728 40.91008, -74.26735 40.910...",250,Morris,Lincoln Park Borough,1416_120.1_26,1416,120.1,26,None,0.146595,...,None,Blue Acres Program,Encumbered,None,None,None,1600812506000,{DB162534-7E0F-4967-B634-B5A5ED3AD67D},353.179036,6385.635110
1,"POLYGON ((-74.36938 40.46128, -74.36983 40.461...",292,Middlesex,Sayreville Borough,1219_169.41_10,1219,169.41,10,None,0.143481,...,None,Blue Acres Program,Encumbered,None,None,None,1600812506000,{D19E45C5-60AC-4B01-B2FA-19578C0763EB},349.999947,6249.996620
2,"POLYGON ((-74.28239 40.61167, -74.28235 40.611...",324,Union,Rahway City,2013_167_20,2013,167,20,None,0.386333,...,None,Blue Acres Program,Encumbered,None,None,None,1600812506000,{140869DC-699C-47CF-8A60-62776DC8AEF6},563.795551,16828.598255
3,"POLYGON ((-74.26964 40.91227, -74.26927 40.912...",349,Morris,Lincoln Park Borough,1416_125_36,1416,125,36,None,0.160836,...,None,Blue Acres Program,Encumbered,None,None,None,1600812506000,{23225755-A351-4C7B-97B7-3C63F998295F},380.924390,7005.977784
4,"POLYGON ((-74.36807 40.46073, -74.36806 40.460...",396,Middlesex,Sayreville Borough,1219_168.02_5,1219,168.02,5,None,0.149220,...,None,Blue Acres Program,Encumbered,None,None,None,1600812506000,{327BC35B-BF22-43D7-92A1-C92859218C99},359.999763,6499.992644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1431,"POLYGON ((-74.59017 40.52982, -74.59050 40.529...",90221,Somerset,Manville Borough,1811_265_14,1811,265,14,None,0.057393,...,Public Access,Blue Acres Program,Encumbered,None,None,None,1651867570000,{19552FCF-111D-4ED4-A8C4-192DE6CEAAB0},250.000123,2500.017930
1432,"POLYGON ((-74.51968 39.38297, -74.51984 39.382...",90511,Atlantic,Pleasantville City,0119_71_80,0119,71,80,None,0.227273,...,None,Blue Acres Program,Encumbered,None,None,None,1591817866000,{B984C813-E8C1-42D9-AF6B-EA6801DA3C90},399.999736,9899.986010
1433,"POLYGON ((-75.23318 39.28453, -75.23305 39.284...",90577,Cumberland,Downe Township,0604_7_21.01,0604,7,21.01,None,0.052446,...,Public Access,Blue Acres Program,Encumbered,Nantuxent Wildlife Management Area,None,None,1630711657000,{C14F9F68-D556-400C-A2B8-8257E6A35C9D},191.698752,2284.535800
1434,"POLYGON ((-75.23328 39.28464, -75.23315 39.284...",90591,Cumberland,Downe Township,0604_7_20.01,0604,7,20.01,None,0.049931,...,Public Access,Blue Acres Program,Encumbered,Nantuxent Wildlife Management Area,None,None,1630711647000,{8938760B-92EF-4A76-86AD-E3439BE4FD8A},187.089890,2174.998674


In [1]:
blueAcres.explore()

NameError: name 'blueAcres' is not defined

In [70]:
count = blueAcres.groupby(["MUNICIPALITY"]).size().sort_values(ascending=True).reset_index().rename(columns={0: 'COUNT'})
count['Rank'] = count['COUNT'].rank(ascending=False)
color_mapping = {1: 'lightblue', 
                 2: 'lightblue', 
                 3: 'lightblue', 
                 'others': 'lightgrey'}
count['Color'] = count['Rank'].map(color_mapping).fillna(color_mapping['others'])

# Plot the bar graph using hvplot
count.hvplot.barh(
    x='MUNICIPALITY', 
    y='COUNT', 
    line_width = 0,
    xlabel='', 
    ylabel='Count',
    title='Properties Purchased by Municipality',
    color = 'Color',
    legend=False,
    height = 600,
    width = 800
)


:Bars   [MUNICIPALITY]   (COUNT,Color)

In [110]:
NJ_block_groups = pygris.block_groups(
    state="NJ", year=2019
).to_crs("EPSG:4326")

join = gpd.sjoin(blueAcres, NJ_block_groups, how = "inner").drop_duplicates(subset='PAMS_PIN')


unique_tracts = join.groupby(["MUNICIPALITY", "TRACTCE"]).size().reset_index()

Using FIPS code '34' for input 'NJ'


In [112]:

unique_tracts = (unique_tracts['0'] < 40)


KeyError: '0'